In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_tweet = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv", na_values = " ")
test_tweet = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")
submission = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/sample_submission.csv")

In [ ]:
from contractions import CONTRACTION_MAP
import unicodedata

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')

import random
random.seed(123)
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
#ignore warnings
import warnings
warnings.filterwarnings('ignore')



In [ ]:
train_tweet.shape, test_tweet.shape

In [ ]:
train_tweet.isnull().sum()

In [ ]:
null_columns=train_tweet.columns[train_tweet.isnull().any()]
print(null_columns)
print(train_tweet[train_tweet["text"].isnull()][null_columns])

In [ ]:
train_tweet.dropna(axis = 0, how ='any',inplace=True)

In [ ]:
test_tweet.isnull().sum()

In [ ]:
train_tweet = train_tweet.drop_duplicates()
test_tweet = test_tweet.drop_duplicates()

In [ ]:
# Distribution of the Sentiment Column
train_tweet['sentiment'].value_counts()

In [ ]:
sns.countplot(x=train_tweet['sentiment'],data=train_tweet)
plt.show()

In [ ]:
# Distribution of the Sentiment Column in percentages
train_tweet['sentiment'].value_counts(normalize=True).plot(kind='bar', title="sentiment distribution in percentages")
plt.show()

In [ ]:
def clean_data(text):
    
    text = ' '.join(map(str, text))
    
    tweet = BeautifulSoup(text).get_text()
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())#remove @ and #tags
    tweet = ' '.join(re.sub("(\w+:\/\/)", " ", tweet).split())#remove urls
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())#remove punctuations
    tweet = tweet.lower()#changing to lowercase
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))#fix misspelled words
    tweet = emoji.demojize(tweet)#emoji to words
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    return tweet
    

In [ ]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
            if contraction_mapping.get(match)\
            else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub(" ` ", "", expanded_text)
    
    return expanded_text

In [ ]:
def process_text(input_string):
    sentences = sent_tokenize(input_string)#sentence tokenization
    listToStr = ' '.join(map(str, sentences))
    tokens = word_tokenize(listToStr)#word tokenization
    lower_tokens = [token.lower() for token in tokens]#case Lowering
    customize_STOPWORDS = ['u', 'ü', 'ur', '4', '2', 'im', 'doin', 'ure']
    meaningful_words = [token for token in lower_tokens if token not in customize_STOPWORDS]#stopwords removal
    #singles = [lmtzr.lemmatize(plural) for plural in meaningful_words]
    #root_verbs = [lmtzr.lemmatize(single,pos ="v") for single in singles]
    #root_adjectives = [lmtzr.lemmatize(root_verb,pos ="a") for root_verb in root_verbs]
        
    return meaningful_words

In [ ]:
train_tweet['text'] = train_tweet.text.apply(expand_contractions)
train_tweet['selected_text'] = train_tweet.selected_text.apply(expand_contractions)
test_tweet['text'] = test_tweet.text.apply(expand_contractions)

In [ ]:
train_tweet['text'] = train_tweet.text.apply(process_text)
train_tweet['selected_text'] = train_tweet.selected_text.apply(process_text)
test_tweet['text'] = test_tweet.text.apply(process_text)


In [ ]:
import emoji
import itertools
train_tweet['text'] = train_tweet.text.apply(clean_data)
train_tweet['selected_text'] = train_tweet.selected_text.apply(clean_data)
test_tweet['text'] = test_tweet.text.apply(clean_data)


In [ ]:
train_tweet.head(), test_tweet.head()

In [ ]:
X = train_tweet
# split X into training and validation sets 
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(X, random_state=1)
print(X_train.shape)
print(X_val.shape)


In [ ]:
pos_train = X_train[X_train['sentiment'] == 'positive']
neutral_train = X_train[X_train['sentiment'] == 'neutral']
neg_train = X_train[X_train['sentiment'] == 'negative']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2,max_features=10000,stop_words='english')

X_train_cv = cv.fit_transform(X_train['text'])

X_pos = cv.transform(pos_train['text'])
X_neutral = cv.transform(neutral_train['text'])
X_neg = cv.transform(neg_train['text'])


pos_count_df = pd.DataFrame(X_pos.toarray(), columns=cv.get_feature_names())
neutral_count_df = pd.DataFrame(X_neutral.toarray(), columns=cv.get_feature_names())
neg_count_df = pd.DataFrame(X_neg.toarray(), columns=cv.get_feature_names())


In [ ]:
# Create dictionaries of the words within each sentiment group, where the values are the proportions of tweets that 
# contain those words

pos_words = {}
neutral_words = {}
neg_words = {}

for k in cv.get_feature_names():
    pos = pos_count_df[k].sum()
    neutral = neutral_count_df[k].sum()
    neg = neg_count_df[k].sum()
    
    pos_words[k] = pos/pos_train.shape[0]
    neutral_words[k] = neutral/neutral_train.shape[0]
    neg_words[k] = neg/neg_train.shape[0]


In [ ]:
# We need to account for the fact that there will be a lot of words used in tweets of every sentiment.  
# Therefore, we reassign the values in the dictionary by subtracting the proportion of tweets in the other 
# sentiments that use that word.

neg_words_adj = {}
pos_words_adj = {}
neutral_words_adj = {}

for key, value in neg_words.items():
    neg_words_adj[key] = neg_words[key] - (neutral_words[key] + pos_words[key])
    
for key, value in pos_words.items():
    pos_words_adj[key] = pos_words[key] - (neutral_words[key] + neg_words[key])
    
for key, value in neutral_words.items():
    neutral_words_adj[key] = neutral_words[key] - (neg_words[key] + pos_words[key])

In [ ]:
def calculate_selected_text(df_row, tol = 0):
    
    tweet = df_row['text']
    sentiment = df_row['sentiment']
    
    if(sentiment == 'neutral'):
        return tweet
    
    elif(sentiment == 'positive'):
        dict_to_use = pos_words_adj # Calculate word weights using the pos_words dictionary
    elif(sentiment == 'negative'):
        dict_to_use = neg_words_adj # Calculate word weights using the neg_words dictionary
        
    words = tweet.split()
    words_len = len(words)
    subsets = [words[i:j+1] for i in range(words_len) for j in range(i,words_len)]
    
    score = 0
    selection_str = '' # This will be our choice
    lst = sorted(subsets, key = len) # Sort candidates by length
    
    
    for i in range(len(subsets)):
        
        new_sum = 0 # Sum for the current substring
        
        # Calculate the sum of weights for each word in the substring
        for p in range(len(lst[i])):
            if(lst[i][p].translate(str.maketrans('','',string.punctuation)) in dict_to_use.keys()):
                new_sum += dict_to_use[lst[i][p].translate(str.maketrans('','',string.punctuation))]
            
        # If the sum is greater than the score, update our current selection
        if(new_sum > score + tol):
            score = new_sum
            selection_str = lst[i]
            #tol = tol*5 # Increase the tolerance a bit each time we choose a selection

    # If we didn't find good substrings, return the whole text
    if(len(selection_str) == 0):
        selection_str = words
        
    return ' '.join(selection_str)

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
tol = 0.001

X_val['predicted_selection'] = ''

for index, row in X_val.iterrows():
    
    selected_text = calculate_selected_text(row, tol)
    
    X_val.loc[X_val['textID'] == row['textID'], ['predicted_selection']] = selected_text

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
X_val['jaccard'] = X_val.apply(lambda x: jaccard(x['selected_text'], x['predicted_selection']), axis = 1)

print('The jaccard score for the validation set is:', np.mean(X_val['jaccard']))


In [ ]:
#Recalculate word weights using the entire training set.


pos_tr = train_tweet[train_tweet['sentiment'] == 'positive']
neutral_tr = train_tweet[train_tweet['sentiment'] == 'neutral']
neg_tr = train_tweet[train_tweet['sentiment'] == 'negative']

In [ ]:
cv = CountVectorizer(max_df=0.95, min_df=2,
                                     max_features=10000,
                                     stop_words='english')

final_cv = cv.fit_transform(train_tweet['text'])

X_pos = cv.transform(pos_tr['text'])
X_neutral = cv.transform(neutral_tr['text'])
X_neg = cv.transform(neg_tr['text'])

pos_final_count_df = pd.DataFrame(X_pos.toarray(), columns=cv.get_feature_names())
neutral_final_count_df = pd.DataFrame(X_neutral.toarray(), columns=cv.get_feature_names())
neg_final_count_df = pd.DataFrame(X_neg.toarray(), columns=cv.get_feature_names())

In [ ]:
pos_words = {}
neutral_words = {}
neg_words = {}

for k in cv.get_feature_names():
    pos = pos_final_count_df[k].sum()
    neutral = neutral_final_count_df[k].sum()
    neg = neg_final_count_df[k].sum()
    
    pos_words[k] = pos/(pos_tr.shape[0])
    neutral_words[k] = neutral/(neutral_tr.shape[0])
    neg_words[k] = neg/(neg_tr.shape[0])

In [ ]:
neg_words_adj = {}
pos_words_adj = {}
neutral_words_adj = {}

for key, value in neg_words.items():
    neg_words_adj[key] = neg_words[key] - (neutral_words[key] + pos_words[key])
    
for key, value in pos_words.items():
    pos_words_adj[key] = pos_words[key] - (neutral_words[key] + neg_words[key])
    
for key, value in neutral_words.items():
    neutral_words_adj[key] = neutral_words[key] - (neg_words[key] + pos_words[key])

In [ ]:
tol = 0.001

for index, row in test_tweet.iterrows():
    
    selected_text = calculate_selected_text(row, tol)
    
    submission.loc[submission['textID'] == row['textID'], ['selected_text']] = selected_text
    


In [ ]:
submission.to_csv('submission.csv', index = False)

In [ ]:
submission.head(30)